In [1]:
import string
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv("kitap_1.csv")
df = df.drop("Unnamed: 0", axis=1)
df = df.dropna()
df.head(5)

,Kitap,YayınEvi,Yazar
0,Körlük,"KIRMIZI KEDİ YAYINEVİ,","Yazar: Jose Saramago,"
1,Momo,"PEGASUS YAYINLARI,","Yazar: Michael Ende,"
2,Şeker Portakalı (Ciltsiz),"CAN YAYINLARI,","Yazar: Jose Mauro De Vasconcelos,"
3,1984,"CAN YAYINLARI,","Yazar: George Orwell,"
4,Fareler ve İnsanlar,"SEL YAYINCILIK,","Yazar: John Steinbeck,"


In [3]:
split = lambda x: x[:-1]
df["YayınEvi"] = df["YayınEvi"].apply(split)

split = lambda x: x[9:-3]
df["Yazar"] = df["Yazar"].apply(split)

In [4]:
def trlower(text):
    result = ""
    for letter in text:
        if letter == "I":
            letter = "ı"
            result += letter
        elif letter == "İ":
            letter = "i"
            result += letter
        else:
            result += letter
    result = result.lower()
    return result

In [5]:
for col in df.columns:
    df[col] = df[col].apply(trlower)

In [6]:
def remove_punctuation(text):
    no_punc = [words for words in text if words not in string.punctuation]
    word_wo_punc = "".join(no_punc)
    return word_wo_punc

In [7]:
for col in df.columns:
    df[col] = df[col].apply(remove_punctuation)

In [8]:
df.head()

,Kitap,YayınEvi,Yazar
0,körlük,kırmızı kedi yayınevi,jose saramago
1,momo,pegasus yayınları,michael ende
2,şeker portakalı ciltsiz,can yayınları,jose mauro de vasconcelos
3,1984,can yayınları,george orwell
4,fareler ve insanlar,sel yayıncılık,john steinbeck


In [9]:
tfidf_matrix = TfidfVectorizer().fit_transform(df["Yazar"])
tfidf_matrix

<31557x13382 sparse matrix of type '<class 'numpy.float64'>'
	with 70970 stored elements in Compressed Sparse Row format>

In [10]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.       , 0.       , 0.3240409, ..., 0.       , 0.       ,
        0.       ],
       [0.       , 1.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.3240409, 0.       , 1.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 1.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 1.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        1.       ]])

In [11]:
indices = pd.Series(df.index, index=df["Kitap"]).drop_duplicates()
indices

Kitap
körlük                                   0
momo                                     1
şeker portakalı ciltsiz                  2
1984                                     3
fareler ve insanlar                      4
                                     ...  
ben tesadüf değilim                  31708
bilgeliğin kapısında kahve molası    31709
rüzgar akıllı                        31710
rudin kod 11z59                      31711
bütüne bütünden aşkla                31712
Length: 31557, dtype: int64

In [12]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
    sim_scores = sim_scores[1:11]
    indices = [i[0] for i in sim_scores]
    return df["Kitap"].iloc[indices]

In [13]:
df.iloc[30,:]

Kitap       bilinmeyen bir kadının mektubu karton kapak
YayınEvi            türkiye iş bankası kültür yayınları
Yazar                                      stefan zweig
Name: 30, dtype: object

In [14]:
gt = get_recommendations("bilinmeyen bir kadının mektubu karton kapak")
df_gt = pd.DataFrame(columns = ["Kitap","Yazar"])

for index in gt.index:
    value = pd.DataFrame(df.iloc[index, 0:3]).T
    df_gt = pd.concat([df_gt, value], axis=0)

df_gt

,Kitap,Yazar,YayınEvi
29,olağanüstü bir gece,stefan zweig,türkiye iş bankası kültür yayınları
30,bilinmeyen bir kadının mektubu karton kapak,stefan zweig,türkiye iş bankası kültür yayınları
43,amok koşucusu,stefan zweig,türkiye iş bankası kültür yayınları
45,bir çöküşün öyküsü,stefan zweig,türkiye iş bankası kültür yayınları
48,korku,stefan zweig,türkiye iş bankası kültür yayınları
63,ay ışığı sokağı,stefan zweig,türkiye iş bankası kültür yayınları
66,mecburiyet,stefan zweig,türkiye iş bankası kültür yayınları
98,lyon’da düğün,stefan zweig,türkiye iş bankası kültür yayınları
105,bir kadının yaşamından yirmi dört saat,stefan zweig,türkiye iş bankası kültür yayınları
123,geçmişe yolculuk,stefan zweig,türkiye iş bankası kültür yayınları


In [15]:
df.iloc[10,:]

Kitap       saatleri ayarlama enstitüsü
YayınEvi               dergah yayınları
Yazar              ahmet hamdi tanpınar
Name: 10, dtype: object

In [16]:
gt = get_recommendations("saatleri ayarlama enstitüsü")
df_gt = pd.DataFrame(columns = ["Kitap","Yazar"])

for index in gt.index:
    value = pd.DataFrame(df.iloc[index, 0:3]).T
    df_gt = pd.concat([df_gt, value], axis=0)

df_gt

,Kitap,Yazar,YayınEvi
40,beş şehir,ahmet hamdi tanpınar,dergah yayınları
71,huzur,ahmet hamdi tanpınar,dergah yayınları
263,mahur beste,ahmet hamdi tanpınar,dergah yayınları
601,ahmet hamdi tanpınar bütün şiirleri,ahmet hamdi tanpınar,dergah yayınları
622,on dokuzuncu asır türk edebiyatı tarihi,ahmet hamdi tanpınar,dergah yayınları
681,sahnenin dışındakiler,ahmet hamdi tanpınar,dergah yayınları
935,hikayeler yaz yağmuruabdullah efendinin rüyaları,ahmet hamdi tanpınar,dergah yayınları
1433,atları bağlayın geceyi burada geçireceğiz,melisa kesmez,iletişim yayınları
1642,mary stuart,stefan zweig,can yayınları
1941,camları kırın kuşlar kurtulsun,fatoş güney,ithaki yayınları
